# Hull White Model Calibration

In this notebook we illustrate Hull White model calibration.

Hull White model parameters are (time-dependent) short rate volatility and (constant) mean reversion. The calibration methods aim at finding suitable values for these model parameters. 

In [ ]:
import sys
sys.path.append('../') # make sure we can access the src/ folder

from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

from src.helpers import bachelier_implied_vol
from src.hull_white_model import HullWhiteModel
from src.swaption import create_swaption
from src.yieldcurve import YieldCurve

## Global Volatility Calibration to ATM Swaption Volatilities

As a first calibration strategy we analyse calibration of short rate volatility to match at-the-money (ATM) Normal implied volatilities for swaptions.

We use some swaption market data as reference values, formulate an optimisation problem and analyse solutions.

For volatility calibration we assume mean reversion is specified already.

In [ ]:
swaption_data = pd.read_csv('../data/swaption_atm_vols.csv', index_col=0)
swaption_data * 1e+4  # in bp

We want to plot the volatility surface to get some better intuition about the shape of the surface.

In [ ]:
def plot_atm_surface_from_data(data, title=None):
    expiries = [ int(e[:-1]) for e in data.index   ]
    swaps    = [ int(s[:-1]) for s in data.columns ]
    X, Y = np.meshgrid(expiries, swaps)
    fig = plt.figure(figsize=(12, 8))
    ax = fig.gca(projection='3d')
    ax.plot_surface(X, Y, data.values.T * 1e+4, cmap=cm.coolwarm, linewidth=0, antialiased=False)
    ax.set_zlim((30.0, 70.0))
    ax.set_xlabel('expiry (y)')
    ax.set_ylabel('swap term (y)')
    ax.set_zlabel('Normal implied volatility (bp)')
    plt.title(title)
    plt.tight_layout()
    plt.show()

plot_atm_surface_from_data(swaption_data, 'Market ATM Volatilities')

### Model-implied ATM Volatility Surface

We want to compare the market ATM volatilities to corresponding implied volatilities from a model. For this step we set up a first Hull White model with initial model parameters and plot the corresponding implied ATM volatilities.

In [ ]:
yield_curve        = YieldCurve(['70y'], [0.03])
mean_reversion     = 0.05
volatility_times   = np.array([ 2.0, 5.0, 10.0, 20.0 ])
volatility_values  = np.array([  50,  50,   50,   50 ]) * 1e-4

model = HullWhiteModel(yield_curve, mean_reversion, volatility_times, volatility_values)

For implied volatility calculation we re-use a method from *HullWhiteModel.ipynb*.

In [ ]:
def model_implied_volatility(model, expiry_term='10y', swap_term='10y', strike='ATM'):
    swaption = create_swaption(expiry_term, swap_term, model.yield_curve, model.yield_curve, strike)
    option = swaption.bond_option_details()
    fwd_price = model.coupon_bond_option(
        option['expiry_time'],
        option['pay_times'],
        option['cash_flows'],
        option['strike_price'],
        option['call_or_put']
        ) / swaption.annuity()
    implied_vol = bachelier_implied_vol(
        fwd_price,
        swaption.fixed_rate(),
        swaption.fairRate(),
        option['expiry_time'],
        swaption.call_or_put(),
        )
    return implied_vol

We also wrap model-implied ATM volatility surface calculation into a function.

In [ ]:
def model_implied_volatility_surface(model, expiry_terms, swap_terms):
    values = np.array([
        [ model_implied_volatility(model, e, s) for s in swap_terms ]
        for e in expiry_terms
    ])
    return pd.DataFrame(values, index=expiry_terms, columns=swap_terms)

model_data = model_implied_volatility_surface(model, swaption_data.index, swaption_data.columns)
display(model_data)


In [ ]:
plot_atm_surface_from_data(model_data, 'Initial Model-implied Volatilities')

Obviously, this does not look like the market volatility surface which we want to match with our model.

### Volatility Calibration

For volatility calibration we specify a set of calibration expiries and calibration swap terms.

In principle, we could try to calibrate to all input data. However, this computationally more expensive and it does not change the overall result of our analysis.

In [ ]:
calibration_expiry_terms = [ '2y', '5y', '10y', '20y' ]
calibration_swap_term    = [ '2y', '5y', '10y', '20y' ]

Note that the calibration expiries and short rate volatility times coincide. This is chosen deliberately; see the discussion of volatility calibration helpers.

For the calibration expiries/swaps terms we can now obtain the reference volatilities.

In [ ]:
reference_volatilities = swaption_data.loc[calibration_expiry_terms, calibration_swap_term]
reference_volatilities

In the next step we formulate an objective function for volatility calibration.

In [ ]:
def global_calibration_objective(sigma):
    model = HullWhiteModel(yield_curve, mean_reversion, volatility_times, sigma)
    model_volatilities = model_implied_volatility_surface(model, calibration_expiry_terms, calibration_swap_term)
    return (model_volatilities - reference_volatilities).values.reshape((-1)) * 1e+4  # difference in bp

We can test the objective function on our initial volatility values.

In [ ]:
global_calibration_objective(volatility_values)

Now, we can optimise for volatility values that minimise the (or a) norm of our objective function.

As optimisation method we use Levenberg-Marquardt algorithm from scipy.

In [ ]:
from scipy.optimize import least_squares

opt = least_squares(global_calibration_objective, volatility_values, method='lm')
opt

We can check the calibrated volatility term structure

In [ ]:
plt.figure(figsize=(8,5))
plt.step(volatility_times, opt.x * 1e+4)
plt.xlabel('time $t$')
plt.ylabel('$\sigma(t)$ (in bp)')
plt.show()

With the calibrated model we can again check the ATM volatility surface.

In [ ]:
model = HullWhiteModel(yield_curve, mean_reversion, volatility_times, opt.x)
model_data = model_implied_volatility_surface(model, swaption_data.index, swaption_data.columns)
plot_atm_surface_from_data(model_data, 'Model-implied Volatilities, a=%.2f' % mean_reversion )

The resulting model-implied volatility surface already looks more like the market volatilities.

We repeat the calibration for 10% mean reversion and zero mean reversion.

In [ ]:
mean_reversion = 0.10
opt = least_squares(global_calibration_objective, volatility_values, method='lm')
display(opt.message)
model = HullWhiteModel(yield_curve, mean_reversion, volatility_times, opt.x)
model_data = model_implied_volatility_surface(model, swaption_data.index, swaption_data.columns)
plot_atm_surface_from_data(model_data, 'Model-implied Volatilities, a=%.2f' % mean_reversion )

In [ ]:
mean_reversion = 0.0001
opt = least_squares(global_calibration_objective, volatility_values, method='lm')
display(opt.message)
model = HullWhiteModel(yield_curve, mean_reversion, volatility_times, opt.x)
model_data = model_implied_volatility_surface(model, swaption_data.index, swaption_data.columns)
plot_atm_surface_from_data(model_data, 'Model-implied Volatilities, a=%.2f' % mean_reversion )

From the above results we conclude that

  - time dependent volatility allows matching volatilities for different expiries,

  - in addition, mean reversion allows to control the slope of ATM volatilities in swap term direction,

  - low (or zero) mean reversion levels allow for a better global match of market ATM volatilities.

## Product-specific Volatility Calibration

As an alternative approach to global calibration we can be more selective in the choice of reference instruments for calibration.

In particular, if we restrict calibration to one reference instrument per expiry time then we can use a bootstrapping approach for model calibration.

This bootstrapping approach is implemented in below calibration function.

In [ ]:
from scipy.optimize import brentq

def model_from_swaptions(european_swaptions, yield_curve, mean_reversion):
    details  = [ s.bond_option_details() for s in european_swaptions ]
    ref_npv  = [ s.npv()                 for s in european_swaptions ]
    ref_vega = [ s.vega()                for s in european_swaptions ]
    #
    volatility_times  = np.array([ d['expiry_time'] for d in details ])
    volatility_values = np.zeros(volatility_times.shape)
    for idx in tqdm(range(len(european_swaptions)), 'Model calibration'):
        def obj(sigma):
            volatility_values[idx:] = sigma
            model = HullWhiteModel(yield_curve, mean_reversion, volatility_times, volatility_values)
            model_npv = model.coupon_bond_option(
                details[idx]['expiry_time'],
                details[idx]['pay_times'],
                details[idx]['cash_flows'],
                details[idx]['strike_price'],
                details[idx]['call_or_put']
                )
            return (model_npv - ref_npv[idx]) / ref_vega[idx]
        vol_guess = european_swaptions[idx].normalVolatility
        sigma_idx = brentq(obj, 0.1*vol_guess, 5.0*vol_guess, xtol=1.0e-6)
        volatility_values[idx] = sigma_idx
    return HullWhiteModel(yield_curve, mean_reversion, volatility_times, volatility_values)

We test the product-specific calibration for a strip of co-terminal European swaptions.

In [ ]:
maturity = 20 # in years
expiry_terms = [ str(e)+'y' for e in range(1, maturity) ]
swap_terms   = [ str(maturity - e)+'y' for e in range(1, maturity) ]
for e, s in zip(expiry_terms, swap_terms):
    print(e + '-' + s)

The swaptions are set up for a fixed strike rate. Alternatively, we could also use ATM strikes for calibration.

In practice, the choice of strike for calibration depends on the market volatilities available. If volatility smile data is available and e.g. a (shifted) SABR model is calibrated and used for smile interpolation then it makes sense to use the strike rate relevant for instrument pricing.

If only ATM volatility data is available then it is more consistent to calibrate to ATM swaptions.

In [ ]:
swaptions = [
    create_swaption(e, s, yield_curve, yield_curve, strike=0.03, normalVolatility=0.01) # flat volatility
    # create_swaption(e, s, yield_curve, yield_curve, strike=0.03, normalVolatility=swaption_data.loc[e,s])
    for e, s in zip(expiry_terms, swap_terms)
]

With the strip of swaptions and a given mean reversion parameter we can now run the calibration and plot resulting short rate volatilities.

In [ ]:
model = model_from_swaptions(swaptions, yield_curve, mean_reversion=0.05)
plt.figure(figsize=(8,5))
plt.step(model.volatility_times, model.volatility_values * 1e+4)
plt.xlabel('time $t$')
plt.ylabel('$\sigma(t)$ (in bp)')
plt.show()

As a result we see a calibrated volatility term structure on an annual time grid.

Similarly as before we can check how mean reversion impacts the calibrated volatilities.

In [ ]:
times = np.linspace(0.0, maturity, 100*maturity + 1)
fig = plt.figure(figsize=(8, 5))
for a in tqdm([0.11, 0.09, 0.07, 0.05, 0.03, 0.01, -0.01, -0.03, -0.05]):
# for a in tqdm([0.11, 0.09, 0.07, 0.05, 0.03, 0.01, -0.01, ]):
    model = model_from_swaptions(swaptions, yield_curve, mean_reversion=a)
    sigma = np.array([ model.sigma(t) for t in times ]) * 1e+4
    plt.plot(times, sigma, label=str('a = %4.2f' % a))
plt.xlabel('time $t$')
plt.ylabel('short rate volatility $\sigma(t)$ (bp)')
plt.legend()
plt.show()


We find that (for flat implied volatilities) mean reversion around 1% yields the least change in model short rate volatilities.